In [97]:
import numpy as np
import pandas as pd
import tensorflow as tf 
import re

from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight

### https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents/data

In [98]:
data = pd.read_csv('/kaggle/input/us-accidents/US_Accidents_March23.csv',nrows=60*1000)

In [99]:
data

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,A-59996,Source2,2,2016-12-31 13:21:54,2016-12-31 13:51:39,34.024395,-117.772278,NaN,NaN,0.01,...,False,False,False,False,False,False,Day,Day,Day,Day
59996,A-59997,Source2,2,2016-12-31 13:19:08,2016-12-31 13:48:49,34.262665,-116.888870,NaN,NaN,0.01,...,False,False,False,False,False,False,Day,Day,Day,Day
59997,A-59998,Source2,3,2016-12-31 13:23:12,2016-12-31 13:52:56,33.760498,-116.301262,NaN,NaN,0.01,...,False,False,False,False,False,False,Day,Day,Day,Day
59998,A-59999,Source2,2,2016-12-31 13:46:34,2016-12-31 14:16:18,33.864384,-118.220535,NaN,NaN,0.00,...,False,False,False,False,False,False,Day,Day,Day,Day


In [100]:
many_missing_values = data.columns[data.isnull().mean() > 0.20]
data = data.drop(many_missing_values,axis=1)

In [101]:
data.isnull().sum()

ID                           0
Source                       0
Severity                     0
Start_Time                   0
End_Time                     0
Start_Lat                    0
Start_Lng                    0
Distance(mi)                 0
Description                  0
Street                       0
City                         0
County                       0
State                        0
Zipcode                      7
Country                      0
Timezone                     7
Airport_Code                 7
Weather_Timestamp          419
Temperature(F)             673
Humidity(%)                869
Pressure(in)               492
Visibility(mi)             896
Wind_Direction             420
Wind_Speed(mph)          11771
Weather_Condition          801
Amenity                      0
Bump                         0
Crossing                     0
Give_Way                     0
Junction                     0
No_Exit                      0
Railway                      0
Roundabo

In [102]:
#because its training dataset and i think fill 10000 with mean dangress
# we have many rows
data = data.dropna(axis=0)

In [103]:
unneeded = ['ID','Country','Zipcode','Street','Airport_Code']
data = data.drop(unneeded,axis=1)

In [104]:
bin_fets = ['Sunrise_Sunset','Civil_Twilight','Nautical_Twilight','Astronomical_Twilight']
for col in bin_fets :
    data[col] = data[col].apply(lambda x: 1 if x=='Day' else 0)

In [105]:
['Start_Time','End_Time']
data['Start_Time'] = data['Start_Time'].apply(lambda x: x[11:16].replace(':',''))
data['End_Time'] = data['End_Time'].apply(lambda x: x[11:16].replace(':',''))
data['Weather_Timestamp'] = data['Weather_Timestamp'].apply(lambda x: x[11:16].replace(':',''))

In [106]:
data['Source'] = data['Source'].apply(lambda x: 1 if x =='Source2' else 0)

In [107]:
nominal_fets = ['City','State','County','Timezone','Wind_Direction','Weather_Condition']
nominal_prefixs = ['C_','ST_','COUN_','TIME_','WIND_','WEATHER']

for col , prefix in zip(nominal_fets,nominal_prefixs):
    dummie = pd.get_dummies(data[col],prefix=prefix)
    data = pd.concat([data,dummie],axis=1)
    data = data.drop(col,axis=1)

In [108]:
names = data['Description'].copy()

In [109]:
stop_words = stopwords.words('english')

def process(name):
    name = re.sub(r'\d+','',name)
    name = name.split()
    name = "".join([word for word in name if word not in stop_words])
    return name

names = names.apply(process)

In [110]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(names)
names = tokenizer.texts_to_sequences(names)
vocal_lengh = len(list(tokenizer.index_word))
max_lengh = np.max(list(map(lambda x: len(x),names))) + 1
print
print("vocal lengh : ",vocal_lengh,"\n    names lengh : ",len(names),"\n        maxlengh : ",max_lengh)

vocal lengh :  21387 
    names lengh :  47503 
        maxlengh :  11


In [111]:
names = pad_sequences(names,maxlen=max_lengh,padding='post')

In [90]:
data = data.drop('Description',axis=1)

In [91]:
y = data['Severity'] - 1 
x = data.drop('Severity',axis=1)

In [92]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [93]:
x.shape
y.unique()
y.value_counts()

Severity
1    27298
2    20144
0       45
3       16
Name: count, dtype: int64

In [94]:
weights = dict(
    enumerate(
        compute_class_weight(
            'balanced',
            classes = y.unique(),
            y=y
        )
    )
)
weights

{0: 0.4350410286467873,
 1: 0.589542791898332,
 2: 263.90555555555557,
 3: 742.234375}

In [119]:
names_train , names_test , x_train , x_test , y_train , y_test = train_test_split(names,x,y,train_size=0.7,random_state=13)

In [120]:
names_train

array([[ 7967,     0,     0, ...,     0,     0,     0],
       [    1,   462,     0, ...,     0,     0,     0],
       [ 1601, 15121,     0, ...,     0,     0,     0],
       ...,
       [    3,   677,     0, ...,     0,     0,     0],
       [    2, 21303,     0, ...,     0,     0,     0],
       [    5, 16588,     0, ...,     0,     0,     0]], dtype=int32)

In [144]:
input_texts = tf.keras.Input(shape=(11,))
embedding_layer = tf.keras.layers.Embedding(
    input_dim = 11,
    output_dim=64
)(input_texts)

flatten_layer = tf.keras.layers.Flatten()(embedding_layer)

inputs_dense = tf.keras.Input(shape=(895,))
layer_1 = tf.keras.layers.Dense(64,activation='relu')(inputs_dense)
layer_2 = tf.keras.layers.Dense(64,activation='relu')(layer_1)


concatenate = tf.keras.layers.concatenate([input_texts,layer_2])
output = tf.keras.layers.Dense(4,activation='softmax')(concatenate)

model = tf.keras.Model(inputs=[input_texts,inputs_dense],outputs=output)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

hist = model.fit(
    [names_train,x_train],
    y_train,
    validation_split = 0.2,
    batch_size=40,
    epochs=30,
    callbacks=tf.keras.callbacks.ReduceLROnPlateau()
)

Epoch 1/30
666/666 [==============================] - 3s 4ms/step - loss: 59.2180 - accuracy: 0.7035 - val_loss: 3.5080 - val_accuracy: 0.8098 - lr: 0.0010
Epoch 2/30
666/666 [==============================] - 2s 3ms/step - loss: 2.1667 - accuracy: 0.8064 - val_loss: 2.0290 - val_accuracy: 0.8316 - lr: 0.0010
Epoch 3/30
666/666 [==============================] - 2s 3ms/step - loss: 1.3215 - accuracy: 0.8132 - val_loss: 1.4826 - val_accuracy: 0.8011 - lr: 0.0010
Epoch 4/30
666/666 [==============================] - 2s 3ms/step - loss: 1.0903 - accuracy: 0.8190 - val_loss: 1.9700 - val_accuracy: 0.8127 - lr: 0.0010
Epoch 5/30
666/666 [==============================] - 2s 3ms/step - loss: 0.9316 - accuracy: 0.8239 - val_loss: 0.9413 - val_accuracy: 0.8408 - lr: 0.0010
Epoch 6/30
666/666 [==============================] - 2s 3ms/step - loss: 0.9364 - accuracy: 0.8276 - val_loss: 1.3623 - val_accuracy: 0.8157 - lr: 0.0010
Epoch 7/30
666/666 [==============================] - 2s 3ms/step - l

In [145]:
model.evaluate([names_test,x_test],y_test)

446/446 [==============================] - 1s 2ms/step - loss: 0.8071 - accuracy: 0.8316


[0.8071063160896301, 0.8315907716751099]

In [ ]:
#inputs_dense = tf.keras.Input(shape=(895))
#layer_1 = tf.keras.layers.Dense(64,activation='relu')(inputs_dense)
#layer_2 = tf.keras.layers.Dense(64,activation='relu')(layer_1)
#layer_3 = tf.keras.layers.Dense(64,activation='relu')(layer_2)

#output = tf.keras.layers.Dense(4,activation='softmax')(layer_3)

#model = tf.keras.Model(inputs=inputs_dense,outputs=output)

#model.compile(
#    optimizer='adam',
#    loss='sparse_categorical_crossentropy',
#    metrics=['accuracy']
#)

#hist = model.fit(
#    x_train,
#    y_train,
#    validation_split = 0.2,
#    batch_size=40,
#    epochs=30,
#    callbacks=tf.keras.callbacks.ReduceLROnPlateau()
#)